In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# BART as ds model and to produce articles given summaries

import os
import random
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import BartForConditionalGeneration
from hyperparams import seed_
from utils import *

ModuleNotFoundError: No module named 'utils'

In [ ]:
from datasets import load_dataset
dataset = load_dataset('cnn_dailymail', '3.0.0')


Reusing dataset cnn_dailymail (C:\Users\kevin\.cache\huggingface\datasets\cnn_dailymail\3.0.0\3.0.0\3cb851bf7cf5826e45d49db2863f627cba583cbc32342df7349dfe6c38060234)
100%|██████████| 3/3 [00:00<00:00, 40.12it/s]


In [ ]:
len(dataset['train']['article'])

287113

In [ ]:
from data_set import *
from BART import *
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler, SubsetRandomSampler
from transformers import BartTokenizer
bart_tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')
bart_criterion = torch.nn.CrossEntropyLoss(ignore_index = bart_tokenizer.pad_token_id, reduction='none')
bart_criterion = bart_criterion.cuda()
bart = BART(bart_criterion,bart_tokenizer)
bart = bart.cuda()

train_data = get_train_Dataset(dataset,5 , bart_tokenizer)# Create the DataLoader for our training set.
train_dataloader = DataLoader(train_data, sampler=RandomSampler(train_data), 
                        batch_size=1, pin_memory=True, num_workers=0)


tensor([0, 1, 2, 3, 4])
Input shape: 
torch.Size([5, 500]) torch.Size([5, 500])
Target shape: 
torch.Size([5, 74]) torch.Size([5, 74])


In [ ]:
bart.encoder.embed_tokens.weight.shape

torch.Size([50265, 768])

In [ ]:
bart.encoder.embed_tokens

Embedding(50265, 768, padding_idx=1)

In [ ]:
bart_tokenizer.encode("reason haha im here is that why im here")

[0, 41995, 46116, 4356, 259, 16, 14, 596, 4356, 259, 2]

In [ ]:
bart_tokenizer('reason haha im here is that why im herereason haha im here is that why im herereason haha im here is that why im herereason haha im here is that why im herereason haha im here is that why im herereason haha im here is that why im herereason haha im here is that why im herereason haha im here is that why im herereason haha im here is that why im herereason haha im here is that why im herereason haha im here is that why im herereason haha im here is that why im herereason haha im here is that why im herereason haha im here is that why im herereason haha im here is that why im herereason haha im here is that why im herereason haha im here is that why im herereason haha im here is that why im herereason haha im here is that why im herereason haha im here is that why im herereason haha im here is that why im herereason haha im here is that why im herereason haha im here is that why im herereason haha im here is that why im here', return_tensors='pt', padding=True, truncation = True, max_length = 10)

{'input_ids': tensor([[    0, 41995, 46116,  4356,   259,    16,    14,   596,  4356,     2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [ ]:
bart_tokenizer.pad_token_id


1

In [ ]:
from torch.autograd import Variable
for step, batch in enumerate(train_dataloader):
    # print("STEP:",step+1,'\tbatch len:',len(batch))
    # # print(batch[-1])

    # Input and its attentions
    article_bart = Variable(batch[0], requires_grad=False).cuda()
    # # print(batch[0])
    article_bart_attn = Variable(batch[1], requires_grad=False).cuda()    
    # print(batch[0].shape)
    output =bart_tokenizer.batch_decode(bart.generate(article_bart))
    # print(len(output[0]))
    # break

STEP: 1 	batch len: 5
torch.Size([1, 500])
1392
STEP: 2 	batch len: 5
torch.Size([1, 500])
2524
STEP: 3 	batch len: 5
torch.Size([1, 500])
1269
STEP: 4 	batch len: 5
torch.Size([1, 500])
1543
STEP: 5 	batch len: 5
torch.Size([1, 500])
2362


In [ ]:
from transformers import pipeline,T5ForConditionalGeneration,T5Tokenizer
model = T5ForConditionalGeneration.from_pretrained('t5-base')
t5_tokenizer = T5Tokenizer.from_pretrained('t5-base')

Downloading: 100%|██████████| 773k/773k [00:00<00:00, 1.90MB/s]
Downloading: 100%|██████████| 1.32M/1.32M [00:00<00:00, 2.79MB/s]


In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
tokenizer = T5Tokenizer.from_pretrained("t5-small")
model = T5ForConditionalGeneration.from_pretrained("t5-small")

model(t5_tokenizer("hi im kevin"))


In [ ]:
model(inputs_embeds = [1])

AttributeError: 'list' object has no attribute 'size'

In [ ]:
model.encoder.parameters

<bound method Module.parameters of T5Stack(
  (embed_tokens): Embedding(32128, 512)
  (block): ModuleList(
    (0): T5Block(
      (layer): ModuleList(
        (0): T5LayerSelfAttention(
          (SelfAttention): T5Attention(
            (q): Linear(in_features=512, out_features=512, bias=False)
            (k): Linear(in_features=512, out_features=512, bias=False)
            (v): Linear(in_features=512, out_features=512, bias=False)
            (o): Linear(in_features=512, out_features=512, bias=False)
            (relative_attention_bias): Embedding(32, 8)
          )
          (layer_norm): T5LayerNorm()
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (1): T5LayerFF(
          (DenseReluDense): T5DenseReluDense(
            (wi): Linear(in_features=512, out_features=2048, bias=False)
            (wo): Linear(in_features=2048, out_features=512, bias=False)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (layer_norm): T5LayerNorm()
   

In [ ]:

from transformers import BartForConditionalGeneration

In [ ]:
input_ids = tokenizer('translate English to French: I am doing project on Thanksgiving.I am doing project on Thanksgiving.I am doing project on Thanksgiving.I am doing project on Thanksgiving.I am doing project on Thanksgiving.I am doing project on Thanksgiving.I am doing project on Thanksgiving.I am doing project on Thanksgiving.', return_tensors='pt').input_ids
outputs = model.generate(input_ids)
# print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Je fais un projet à l'occasion de l'Action d'


In [ ]:
help(model.generate)

Help on method generate in module transformers.generation_utils:

generate(input_ids: Union[torch.LongTensor, NoneType] = None, max_length: Union[int, NoneType] = None, min_length: Union[int, NoneType] = None, do_sample: Union[bool, NoneType] = None, early_stopping: Union[bool, NoneType] = None, num_beams: Union[int, NoneType] = None, temperature: Union[float, NoneType] = None, top_k: Union[int, NoneType] = None, top_p: Union[float, NoneType] = None, repetition_penalty: Union[float, NoneType] = None, bad_words_ids: Union[Iterable[int], NoneType] = None, bos_token_id: Union[int, NoneType] = None, pad_token_id: Union[int, NoneType] = None, eos_token_id: Union[int, NoneType] = None, length_penalty: Union[float, NoneType] = None, no_repeat_ngram_size: Union[int, NoneType] = None, encoder_no_repeat_ngram_size: Union[int, NoneType] = None, num_return_sequences: Union[int, NoneType] = None, max_time: Union[float, NoneType] = None, max_new_tokens: Union[int, NoneType] = None, decoder_start_tok

In [ ]:
# dataset = load_dataset('opus100','en-fr')#large
dataset = load_dataset('opus_euconst','en-fr')#small


Reusing dataset opus_euconst (C:\Users\kevin\.cache\huggingface\datasets\opus_euconst\en-fr\1.0.0\d1e611a011f28fdda67a97024820e0a3813b4e4decca194d9a20b3207a39b908)
100%|██████████| 1/1 [00:00<00:00, 501.41it/s]


In [ ]:
dataset

DatasetDict({
    test: Dataset({
        features: ['translation'],
        num_rows: 2000
    })
    train: Dataset({
        features: ['translation'],
        num_rows: 1000000
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 2000
    })
})

In [ ]:
dataset["train"][:20]

{'translation': [{'en': 'The time now is 05:08 .',
   'fr': 'The time now is 05:05 .'},
  {'en': 'This Regulation shall enter into force on the seventh day following its publication in the Official Journal of the European Union.',
   'fr': "Le présent règlement entre en vigueur le septième jour suivant celui de sa publication au Journal officiel de l'Union européenne."},
  {'en': "Hello, what's that?", 'fr': "Qu'est-ce que c'est que ça ?"},
  {'en': 'And then I will teach you everything i know.',
   'fr': "Et alors, je t'apprendrai tout ce que je sais."},
  {'en': 'Did you find something?', 'fr': 'Par ici !'},
  {'en': 'Article 6', 'fr': 'Article 6'},
  {'en': "Oh, honey, it's not your fault.",
   'fr': "- Tu n'es pas responsable."},
  {'en': "I'm onto him now.", 'fr': 'Je le tiens.'},
  {'en': "DG XVI's Internet site (electronic address: http://www.cec.lu/en/comm/dg16/dg16home.html) now contains detailed information in English on pilot projects on innovation, the Information society, 

In [ ]:
import numpy as np

x = np.zeros((20,20))
# print(x)
x[:,:15].shape

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.

(20, 15)